<a href="https://colab.research.google.com/github/eriksali/DNN_2023_NLP/blob/main/NLP13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP Class Lecture 13 Code Examples

Oakland University, W 23, Prof. Wilson

## Simple NN with pytorch

In [ ]:
import torch
import torch.nn as nn

In [ ]:
model = torch.nn.Sequential( nn.Linear(2,2,bias=False), nn.Linear(2,1,bias=False), nn.Sigmoid())

In [ ]:
with torch.no_grad():
    model[0].weight[0,0] = .3 
    model[0].weight[1,0] = -.1 
    model[0].weight[0,1] = .1 
    model[0].weight[1,1] = .2 
    model[1].weight[0,0] = .1 
    model[1].weight[0,1] = -.05 

In [ ]:
loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [ ]:
model.forward(x)

In [ ]:
x = torch.Tensor([2,1])
h = model[0](x)
z = model[1](h)
o = model[2](z)
out = o
print(x)
print(model[0].weight)
print(h)
print(z)
print(o)

tensor([2., 1.])
Parameter containing:
tensor([[ 0.3000,  0.1000],
        [-0.1000,  0.2000]], requires_grad=True)
tensor([0.7000, 0.0000], grad_fn=<SqueezeBackward3>)
tensor([0.0700], grad_fn=<SqueezeBackward3>)
tensor([0.5175], grad_fn=<SigmoidBackward0>)


In [ ]:
loss = loss_fn(out, torch.Tensor([1.0]))
loss

tensor(0.6588, grad_fn=<BinaryCrossEntropyBackward0>)

In [ ]:
loss.backward()

In [ ]:
print('layer 2 gradient:',model[1].weight.grad)
print('layer 1 gradient:',model[0].weight.grad)

layer 2 gradient: tensor([[-0.3378,  0.0000]])
layer 1 gradient: tensor([[-0.0965, -0.0483],
        [ 0.0483,  0.0241]])


In [ ]:
optimizer.step()
print('new layer 1:',model[0].weight)
print("new layer 2:",model[1].weight) 

new layer 1: Parameter containing:
tensor([[ 0.3001,  0.1000],
        [-0.1000,  0.2000]], requires_grad=True)
new layer 2: Parameter containing:
tensor([[ 0.1003, -0.0500]], requires_grad=True)


### Scratch space

In [ ]:
import numpy as np
def logistic(z, derivative=False):
    if not derivative:
        return 1 / (1 + np.exp(-z))
    else:
        return logistic(z) * (1 - logistic(z))
logistic(-.3153,True)

0.2438881376106578

In [ ]:
t3 = torch.Tensor([0.6,0])
sm = torch.softmax(t3,dim=0)
.6457*.1 + .3543*-0.05
torch.sigmoid(torch.Tensor([.046855]))

In [ ]:
def getBack(var_grad_fn):
    print(var_grad_fn)
    for n in var_grad_fn.next_functions:
        if n[0]:
            try:
                tensor = getattr(n[0], 'variable')
                print(n[0])
                print('Tensor with grad found:', tensor)
                print(' - gradient:', tensor.grad)
                print()
            except AttributeError as e:
                getBack(n[0])

getBack(loss.grad_fn)

## Word embeddings

In [ ]:
! pip install --upgrade gensim

In [ ]:
import gensim
gensim.__version__

'4.3.0'

In [ ]:
# download pretrained embeddings

import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

In [ ]:
for index, word in enumerate(wv.index_to_key):
    if index == 10:
        break
    print(f"word #{index}/{len(wv.index_to_key)} is {word}")

In [ ]:
vec_king = wv['king']
print(vec_king)

In [ ]:
pairs = [
    ('car', 'minivan'),   # a minivan is a kind of car
    ('car', 'bicycle'),   # still a wheeled vehicle
    ('car', 'airplane'),  # ok, no wheels, but still a vehicle
    ('car', 'cereal'),    # ... and so on
    ('car', 'communism'),
]
for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2, wv.similarity(w1, w2)))

In [ ]:
print(wv.most_similar(positive=['car', 'minivan'], topn=5))

In [ ]:
print(wv.doesnt_match(['fire', 'water', 'land', 'sea', 'air', 'car']))

In [ ]:
print(wv.most_similar(positive=['king', 'woman'], negative=['man'], topn=5))